# Evaluate ensemble members

The goal of this script is to identify ensemble members that meet specific ecological criteria

In [24]:
import pandas as pd
import numpy as np
import os
import esm_tools
pd.set_option('display.max_rows', 500) 
import shutil

In [25]:
# Path to where the prior ensemble run parameter files are stored
path_to_ensemble_param_files_root = '/glade/u/home/adamhb/ahb_params/fates_api_25/ensembles/'

In [26]:
def get_most_recent_file_in_dir(folder_path):

    # Get a list of all files in the folder
    files = os.listdir(folder_path)

    # Filter out only files (excluding subdirectories)
    files = [file for file in files if os.path.isfile(os.path.join(folder_path, file))]

    # Sort the files by creation time (most recent first)
    files.sort(key=lambda x: os.path.getctime(os.path.join(folder_path, x)), reverse=True)

    # Check if there are any files in the folder
    if files:
        most_recent_file = files[0]
        print(f"The most recently created file is: {most_recent_file}")
        return most_recent_file
    else:
        print("The folder is empty.")

## Set up which files are being aggregated and evaluated

In [48]:
case_name_prefix = 'CZ2_equilibrium_101723_'
case_name_suffix = '_-17e2acb6a_FATES-8a054a12'
case_numbers = [1,2,3]

all_member_output_filename = 'all_member_output_CZ2_equilibrium_101723.csv'
successful_members_file_name = "successfull_members_CZ2_equilibrium_101723.csv"

# Path to put any processed output
processed_output_root = '/glade/scratch/adamhb/processed_output'

case_tags = [str(case_num).rjust(2, '0') for case_num in case_numbers]
print("Case Tags:")
print(case_tags)
print("\n")
folder_names = [case_name_prefix + case_tag + case_name_suffix for case_tag in case_tags]
folder_paths = [os.path.join(processed_output_root,folder_name) for folder_name in folder_names]

print("Folders:")
print(folder_paths)
print("\n")

print("Files")
[get_most_recent_file_in_dir(folder_path) for folder_path in folder_paths]
print("\n")

print("Full file paths:")
files = [get_most_recent_file_in_dir(folder_path) for folder_path in folder_paths]
full_file_paths = [os.path.join(folder_paths[i],files[i]) for i in range(len(files))]
print(full_file_paths)

Case Tags:
['01', '02', '03']


Folders:
['/glade/scratch/adamhb/processed_output/CZ2_equilibrium_101723_01_-17e2acb6a_FATES-8a054a12', '/glade/scratch/adamhb/processed_output/CZ2_equilibrium_101723_02_-17e2acb6a_FATES-8a054a12', '/glade/scratch/adamhb/processed_output/CZ2_equilibrium_101723_03_-17e2acb6a_FATES-8a054a12']


Files
The most recently created file is: ensemble_output_CZ2_equilibrium_101723_01_-17e2acb6a_FATES-8a054a12_decadal_metrics.csv
The most recently created file is: ensemble_output_CZ2_equilibrium_101723_02_-17e2acb6a_FATES-8a054a12_decadal_metrics.csv
The most recently created file is: ensemble_output_CZ2_equilibrium_101723_03_-17e2acb6a_FATES-8a054a12_decadal_metrics.csv


Full file paths:
The most recently created file is: ensemble_output_CZ2_equilibrium_101723_01_-17e2acb6a_FATES-8a054a12_decadal_metrics.csv
The most recently created file is: ensemble_output_CZ2_equilibrium_101723_02_-17e2acb6a_FATES-8a054a12_decadal_metrics.csv
The most recently created file is:

## Get all data into on dataframe

In [42]:
df = pd.DataFrame()
i = 0
for f in full_file_paths:
    df_tmp = pd.read_csv(f)
    df_tmp = df_tmp.drop(df_tmp.columns[0], axis=1)
    df_tmp['case_tag'] = case_tags[i]
    df = pd.concat([df,df_tmp],axis = 0)
    i = i + 1

In [37]:
df.to_csv(processed_output_root + "/" + "with_fire_metrics_" + all_member_output_filename)

In [45]:
cols = ["case_tag","inst","BA","BA_pine","BA_cedar","BA_fir","BA_shrub","BA_oak",
 "AGB","TreeStemD","ResproutD_oak","ResproutD_shrub","ShannonE","NPP","FailedPFTs",
        "Pct_shrub_cover_canopy","Pct_shrub_cover","Combustible_fuel","Burned_area","Pct_high_severity_1700","Pct_high_severity_3500"]

In [39]:
df = df[cols]

In [46]:
df[cols]

,case_tag,inst,BA,BA_pine,BA_cedar,BA_fir,BA_shrub,BA_oak,AGB,TreeStemD,...,ResproutD_shrub,ShannonE,NPP,FailedPFTs,Pct_shrub_cover_canopy,Pct_shrub_cover,Combustible_fuel,Burned_area,Pct_high_severity_1700,Pct_high_severity_3500
0,01,3,23.958220,0.295358,15.511509,0.070217,4.297803,3.783331,5.462409,275.145955,...,0.000000,0.591781,0.651599,1,0.106477,0.189698,0.556339,0.013,0.000000,0.000000
1,01,11,24.391005,0.074526,16.717539,0.985907,6.075087,0.537947,4.456143,549.873896,...,0.098707,0.519827,0.833952,1,0.136743,0.438737,0.834401,0.039,4.054054,0.000000
2,01,14,18.205610,0.074029,5.964147,0.303579,0.535164,11.328691,4.697561,800.293684,...,0.000000,0.531310,0.808131,1,0.000065,0.043620,0.853940,0.026,28.813559,1.694915
3,01,19,16.377903,0.001866,4.996992,0.176686,2.995747,8.206612,3.789244,264.937449,...,0.000000,0.664239,0.731382,1,0.097210,0.172463,1.436287,0.016,50.434783,10.434783
4,01,21,21.933979,6.421227,9.719576,1.753197,0.242180,3.797798,5.086062,379.617140,...,0.000000,0.792590,0.575475,0,0.020254,0.022734,0.426593,0.043,0.000000,0.000000
5,01,22,23.984100,0.574372,8.991769,8.577020,0.400098,5.440842,6.969836,385.876298,...,0.000000,0.764074,0.599534,0,0.015976,0.017849,0.719824,0.022,0.000000,0.000000
6,01,23,22.735706,9.061029,0.953378,0.101761,12.590818,0.028721,3.792508,109.073743,...,0.223199,0.534068,0.745532,1,0.409533,0.527732,0.778930,0.085,2.298851,0.000000
7,01,24,17.974762,0.010423,1.317839,0.856292,0.847386,14.942822,6.631061,316.578262,...,0.000000,0.396716,0.887236,1,0.011766,0.041302,0.701013,0.016,36.842105,1.754386
0,02,7,24.086369,15.361878,0.283073,0.202542,4.126918,4.111957,5.975501,649.154708,...,0.000000,0.610958,0.818854,0,0.233132,0.314411,1.912418,0.021,48.101266,11.392405
1,02,15,42.443077,4.479413,36.863630,0.308961,0.789293,0.001785,12.059237,483.535454,...,0.000000,0.292087,0.816989,1,0.019316,0.038751,1.349837,0.008,25.609756,0.000000


In [44]:
list(df.columns)

['BA',
 'AGB',
 'TreeStemD',
 'ResproutD_oak',
 'ResproutD_shrub',
 'ShannonE',
 'NPP',
 'FailedPFTs',
 'Pct_shrub_cover_canopy',
 'Pct_shrub_cover',
 'Combustible_fuel',
 'Burned_area',
 'Pct_high_severity_1700',
 'Pct_high_severity_3500',
 'BA_pine',
 'BA_cedar',
 'BA_fir',
 'BA_shrub',
 'BA_oak',
 'inst',
 'fates_recruit_seed_alloc_mature_1',
 'fates_recruit_seed_alloc_mature_2',
 'fates_recruit_seed_alloc_mature_3',
 'fates_recruit_seed_alloc_mature_4',
 'fates_recruit_seed_alloc_mature_5',
 'fates_recruit_seed_dbh_repro_threshold_1',
 'fates_recruit_seed_dbh_repro_threshold_2',
 'fates_recruit_seed_dbh_repro_threshold_3',
 'fates_recruit_seed_dbh_repro_threshold_5',
 'fates_fire_alpha_SH_1',
 'fates_fire_alpha_SH_2',
 'fates_fire_alpha_SH_3',
 'fates_fire_alpha_SH_4',
 'fates_fire_alpha_SH_5',
 'fates_fire_bark_scaler_1',
 'fates_fire_bark_scaler_2',
 'fates_fire_bark_scaler_3',
 'fates_fire_bark_scaler_4',
 'fates_fire_bark_scaler_5',
 'fates_fire_crown_kill_1',
 'fates_fire_crow

## Filter to ensemble members that meet ecological criteria

CZ2 Pre-Euro-American Management--criteria for analyzing fire regimes

1. There are 0 failing pfts (all pfts have above at least 0.1 m2 ha-1 of basal area)
2. Shrub percent cover is > 1% and < 50%

In [49]:
CZ2_analyze_fire_regime_filter = (df['FailedPFTs'] < 2) & (df['Pct_shrub_cover'] > 0.01) & (df['Pct_shrub_cover_canopy'] < 0.5)
CZ2_analyze_fire_regime = df.loc[CZ2_analyze_fire_regime_filter].sort_values("ShannonE",ascending = False)
CZ2_analyze_fire_regime.shape


#Take just the top 108 members to run a 3-node simulation to equilibrium
#CZ2_analyze_fire_regime = CZ2_analyze_fire_regime[:108]

CZ2_analyze_fire_regime.to_csv(os.path.join(processed_output_root,successfull_members_file_name))

NameError: name 'successfull_members_file_name' is not defined

In [50]:
CZ2_analyze_fire_regime.to_csv("/glade/scratch/adamhb/processed_output/successfull_members_CZ2_equilibrium_101723.csv")

In [23]:
print(list(CZ2_analyze_fire_regime["inst"]))

[21, 22, 19, 8, 7, 3, 12, 23, 14, 11, 31, 7, 23, 24, 15]


## Prepare passing parameterizations for next simulation

In [85]:
new_case_tags

NameError: name 'new_case_tags' is not defined

In [92]:
def aggregate_passing_paramsets(pass_case_tags,passing_inst_tags,n_new_cases,n_inst_per_case,new_subdirs_prefix,
                                path_to_ensemble_param_files_root,old_param_subdir_prefix,new_param_file_base_name):
    
    
    for n in range(n_new_cases):
        new_param_subdir_suffix = str(n+1).rjust(2, '0')
        
        new_param_subdir = os.path.join(path_to_ensemble_param_files_root,new_subdirs_prefix) + "_" + new_param_subdir_suffix
        os.makedirs(new_param_subdir,exist_ok=True)
    
    tmp_inst = 0
    for case_tag,inst_tag in zip(pass_case_tags,passing_inst_tags):
        
        tmp_inst = tmp_inst + 1
        
        param_files_subdir = old_param_subdir_prefix + "_" + case_tag
        
        # Get tag
        inst_tag_with_leading_zeroes = str(inst_tag).rjust(4, '0')
    
        # Get param file with inst tag
        ref_nc_file = esm_tools.find_files_with_substring(directory=os.path.join(path_to_ensemble_param_files_root,
                                                                       param_files_subdir),
                                                substring=inst_tag_with_leading_zeroes)

        # Get full path of originl param file
        ref_nc_file_full_path = os.path.join(path_to_ensemble_param_files_root,param_files_subdir,ref_nc_file[0])

        print("originl_file:",ref_nc_file_full_path)

        
        #Copy to new folder
        os.makedirs("tmp",exist_ok=True)
        
        new_tag = str(tmp_inst).rjust(4, '0')
    
        new_name = new_param_file_base_name + "_" + new_tag + ".nc"

        dst_file = os.path.join("tmp",new_name)
        
        print("tmp file:",dst_file)
        print("\n")

        shutil.copy(ref_nc_file_full_path,dst_file)
        
        
        # allocate across new cases
        new_case_tags = [str(case_tag+1).rjust(2, '0') for case_tag in range(n_new_cases)]
      
        
    for i,new_case_tag in enumerate(new_case_tags):
            
            new_param_subdir = os.path.join(path_to_ensemble_param_files_root,new_subdirs_prefix) + "_" + new_case_tag
            
            tmp_inst = i * n_inst_per_case
            
            for i in range(n_inst_per_case):
                
                tmp_inst = tmp_inst + 1
                
                tmp_inst_tag = str(tmp_inst).rjust(4, '0')
                
                 # Get param file with inst tag
                ref_nc_file = esm_tools.find_files_with_substring(directory="tmp",
                                                substring=tmp_inst_tag)

                # Get full path of originl param file
                ref_nc_file_full_path = os.path.join("tmp",ref_nc_file[0])
                
                print("tmp_file:",ref_nc_file_full_path)
                
                new_tag = str(i + 1).rjust(4, '0')
                new_name = new_param_file_base_name + "_" + new_tag + ".nc"
                dst_file = os.path.join(path_to_ensemble_param_files_root,new_param_subdir,new_name)
                
                shutil.copy(ref_nc_file_full_path,dst_file)
                
                print("destination:",dst_file)    

In [93]:
aggregate_passing_paramsets(pass_case_tags = CZ2_analyze_fire_regime['case_tag'],
                            passing_inst_tags = CZ2_analyze_fire_regime['inst'],
                            n_new_cases = 3,
                            n_inst_per_case = 36,
                            new_subdirs_prefix = "CZ2_equilibrium_101723",
                            path_to_ensemble_param_files_root = path_to_ensemble_param_files_root,
                            old_param_subdir_prefix = 'ca_5pfts_20cases_4320inst_101223',
                            new_param_file_base_name = 'CZ2_equilibrium_101723')

originl_file: /glade/u/home/adamhb/ahb_params/fates_api_25/ensembles/ca_5pfts_20cases_4320inst_101223_04/ca_5pfts_100523_0036.nc
tmp file: tmp/CZ2_equilibrium_101723_0001.nc


originl_file: /glade/u/home/adamhb/ahb_params/fates_api_25/ensembles/ca_5pfts_20cases_4320inst_101223_13/ca_5pfts_100523_0134.nc
tmp file: tmp/CZ2_equilibrium_101723_0002.nc


originl_file: /glade/u/home/adamhb/ahb_params/fates_api_25/ensembles/ca_5pfts_20cases_4320inst_101223_03/ca_5pfts_100523_0121.nc
tmp file: tmp/CZ2_equilibrium_101723_0003.nc


originl_file: /glade/u/home/adamhb/ahb_params/fates_api_25/ensembles/ca_5pfts_20cases_4320inst_101223_05/ca_5pfts_100523_0092.nc
tmp file: tmp/CZ2_equilibrium_101723_0004.nc


originl_file: /glade/u/home/adamhb/ahb_params/fates_api_25/ensembles/ca_5pfts_20cases_4320inst_101223_04/ca_5pfts_100523_0005.nc
tmp file: tmp/CZ2_equilibrium_101723_0005.nc


originl_file: /glade/u/home/adamhb/ahb_params/fates_api_25/ensembles/ca_5pfts_20cases_4320inst_101223_07/ca_5pfts_100523